In [1]:
import math
from conveyance import cross_section, belt_capacity, conveyor_resistances, power_requirements

g = 9.81  # Gravity

In [2]:
# Cross-sectional area of material on belt
B = 1.2  # Total width of belt
b = 1.03  # Width of max material on belt
l3 = 0.436  #  Width of the idler (3 roll set)
ia = 45  # Idlers angle
sa = 20  # Material surcharge angle, coal

s = cross_section.belt_ca(l3=l3, b=b, ia=ia, sa=sa)
print(f"s:\t{s:.3f} m^2")

s:	0.180 m^2


In [3]:
# Resistances
v = 4.8  # Belt speed (m/s)
p = 0.85  # Density, coal (t/m^3)
v_0 = 0  # Speed of conveyed material (m/s)

# q_vt: Theoretical flow rate,
q_vt = cross_section.belt_capacity(belt_ca=s, v=v)  # Capacity, theoretical (m^3/s)
print(f"q_vt:\t{q_vt:.3f} m^3/s")

# q_mt: Theoretical capacity, mass per m  (kg/m)
q_mt = belt_capacity.mass_density_material(v=v, q_v=q_vt, p=p)
print(f"q_mt:\t{q_mt:.0f} kg/m")

# q_t: Theoretical capacity, mass per hour  (t/h)
q_vt = 0.864  # Set to 0.864
q_t = q_vt * (p * 1000) * (3600 / 1000)
print(f"q_t:\t{q_t:.2f} t/h")

# q_m: Actual capacity, mass per m (kg/m)
q = 2300  # Capacity, assumed (87% of q_vt)
q_m = belt_capacity.mass_density_material(v, q=q)
print(f"q_m:\t{q_m:.1f} kg/m")

# q_v: Actual capacity, volume per second (m^3/s)
q_v = belt_capacity.volume_carried_material(q=q, p=p)
print(f"q_v:\t{q_v:.4f} m^3/s")

q_vt:	0.865 m^3/s
q_mt:	153 kg/m
q_t:	2643.84 t/h
q_m:	133.1 kg/m
q_v:	0.7516 m^3/s


In [4]:
a_o = 1.2  # Carry idler spacing (m)
m_o = 15.5  # Carry idler mass (kg)

a_u = 3.0  # Return idler spacing (m)
m_u = 13.2  # Return idler mass (kg)

# q_ro: Mass of revolving idler parts on carry side, kg/m
q_ro = belt_capacity.mass_density_idler(a_o, m_o)
print(f"q_ro:\t{q_ro:.2f} kg/m")

# q_ru: Mass of revolving idler parts on return side, kg/m
q_ru = belt_capacity.mass_density_idler(a_u, m_u)
print(f"q_ru:\t{q_ru:.2f} kg/m")

q_ro:	12.92 kg/m
q_ru:	4.40 kg/m


In [5]:
q_b = 16.44  # Belt mass (kg/m)
install_a = 0  # Installation angle of the conveyor (deg)
conveyor_length = 143  # Center-to-centre length of the conveyor
ff = 0.02  # Artificial friction factor (average operating conditions)

# f_h: Calculate the main resistance (Fh)
f_h = conveyor_resistances.resistance_main(q_m, q_b, q_ro, q_ru, conveyor_length, install_a, ff)
print(f"f_h:\t{f_h:.2f} N")

# f_st: Calculate the gravity forces (Fst)
H = 0  # Conveyor lift (m)
f_st = conveyor_resistances.resistance_gravity(q_m, H)
print(f"f_st:\t{f_st:.1f} N")

f_h:	5142.73 N
f_st:	0.0 N


In [6]:
# Calculate the secondary resistance

# f_ba: Inertial and friction resistance at the loading point and
# in the acceleration area between the material handled and the belt (FbA)
f_ba = conveyor_resistances.resistance_inertial_friction(q_v=q_v, p=p, v=v, v_0=v_0)
print(f"f_ba:\t{f_ba:.1f} N")

mu1 = 0.5  # Coefficients between material/belt
mu2 = 0.7  # Coefficients between material/skirtplates
b1 = 0.75  # Width between skirtplates

# f_f: Resistance between handled material and skirtplates in acceleration area (Ff)
f_f = conveyor_resistances.resistance_material_acceleration(q_v=q_v, p=p, v=v, v_0=v_0,
                                                            b1=b1, mu1=mu1, mu2=mu2)
print(f"f_f:\t{f_f:.2f} N")

wrap_a = 180
alpha_1 = 180 - wrap_a

# Wrap resistance between the belt and the pulleys (F1t)
f_1t = conveyor_resistances.resistance_belt_wrap(B=B, wrap_a=wrap_a)
print(f"f_1t:\t{f_1t:.0f} N")

# f_n: Calculate Secondary resistance (FN)
# Note: (2 * f_1t) occurs because the tension on the  head (F1) and tail (Ft) pulleys
f_n = f_ba + f_f + (2 * f_1t)
print(f"f_n:\t{f_n:.2f} N")

f_ba:	3066.7 N
f_f:	2390.38 N
f_1t:	360 N
f_n:	6177.05 N


In [7]:
# Calculate the special resistance (Fs)
l_s = 4.8  # Length of installation fitted with skirtplates

# f_gl: Resistance due to friction between the material handled and skirt plates
f_gl = conveyor_resistances.resistance_material_skirtplates(q_v=q_v, p=p, v=v, l_s=l_s, b1=b1, mu2=mu2)
print(f"f_gl:\t{f_gl:.2f} N")

# f_rc: Friction resistance due to belt cleaners fitted to the conveyor
mu3 = 0.6  # Friction coefficient between belt and cleaner
bc_w = 1.2  # Belt cleaner width (m)
bc_t = 8 / 1000  # Belt cleaner thickness (m)
bc_p = 3 * 10 ** 4  # Pressure between cleaner and belt
bc_n = 4  # Number of belt cleaners (three at head end, one at tail end)

f_rc = conveyor_resistances.resistance_belt_cleaners(bc_w=bc_w, bc_t=bc_t, bc_p=bc_p,
                                                     bc_n=bc_n, mu3=mu3)
print(f"f_rc:\t{f_rc:.1f} N")

# f_ep: Resistance due to idler tilting
f_ep = 0

# f_a: Resistance due to friction at a discharge plough
f_a = 0  # No discharge ploughs present

# f_s: Special resistance
f_s = f_ep + f_gl + f_rc + f_a
print(f"f_s:\t{f_s:.2f} N")

f_gl:	1221.34 N
f_rc:	691.2 N
f_s:	1912.54 N


In [8]:
# Calculate the belt sag tension forces
# Note: Is the minimum tensile force to limit belt sag between 2 sets of idlers

h_a_o = 0.01  # Allowable belt sag, carry side
h_a_u = 0.02  # Allowable belt sag, return side

f_bs_min_o, f_bs_min_u = conveyor_resistances.resistance_belt_sag_tension(q_m=q_m, q_b=q_b, a_o=a_o, a_u=a_u,
                                                                          h_a_o=h_a_o, h_a_u=h_a_u)

# f_bs_min_o: Min tensile force to prevent belt sag, carry side
print(f"f_bs_min_o:\t{f_bs_min_o:.0f} N")

# f_bs_min_u: Min tensile force to prevent belt sag, return side
print(f"f_bs_min_u:\t{f_bs_min_u:.0f} N")

f_bs_min_o:	22005 N
f_bs_min_u:	3024 N


In [9]:
# Calculate the peripheral driving force on driving pulley
f_u = f_h + f_n + f_s + f_st
print(f"f_u:\t{f_u:.2f} N")

f_u:	13232.32 N


In [10]:
# Calculate the power requirements

# p_a: Drive pulley power requirements
p_a = f_u * v
print(f"p_a:\t{p_a / 1000:.2f} kW")

d_eta_1 = 0.95  # Fluid coupling efficiency
d_eta_2 = 0.97  # Gearbox efficiency

# p_m: Drive motor power requirements
p_m = power_requirements.power_requirements_motor(f_u=f_u, v=v, d_eta_1=d_eta_1, d_eta_2=d_eta_2)
print(f"p_m:\t{p_m / 1000:.2f} kW")

p_a:	63.52 kW
p_m:	68.93 kW


In [11]:
# Drive pulley
# Calculate the minimum tensile force (t_2) that must be maintained to transmit f_u
mu_b = 0.3  # Belt/Pulley friction coefficient
_, _, t_d = conveyor_resistances.tension_transmit_min(f_u=f_u, wrap_a=wrap_a, mu_b=mu_b)
print(f"{t_d[0]:.3f} >= {t_d[1]:.3f}, {t_d[2]}")  # Ok

2.566 >= 2.566, True


In [12]:
# Tail pulley
# Calculate the minimum belt tension that should be maintained in order to avoid belt sag
# Set t_2 to f_bs_min_o
t_1, t_2, t_t = conveyor_resistances.tension_transmit_min(f_u=f_u, wrap_a=wrap_a, mu_b=mu_b, t_2_min=f_bs_min_o)

print(f"{t_t[0]:.3f} >= {t_t[1]:.3f}, {t_t[2]}")  # Not satisfied

1.601 >= 2.566, False


In [13]:
d = 0.0125  # Belt thickness (m)

# Drive pulley tension
d_0_d = 0.14  # Diameter of inside bearing
D_d = 0.6  # Drive pulley diameter
m_p_d = 1050  # Drive pulley mass
f_1t_d = conveyor_resistances.resistance_belt_wrap_iso(B, d, D_d, d_0_d, m_p_d, t_1, t_2)
print(f"f_1_td:\t{f_1t_d:.0f} N")

# Tail pulley tension
d_0_t = 0.12  # Diameter of inside bearing
D_t = 0.5  # Tail pulley diameter
m_p_t = 800  # Tail pulley mass
f_1t_t = conveyor_resistances.resistance_belt_wrap_iso(B, d, D_t, d_0_t, m_p_t, t_2, t_2)
print(f"f_1_tt:\t{f_1t_t:.0f} N")

f_1_td:	153 N
f_1_tt:	141 N


In [14]:
# f_n: Secondary resistance
f_n = conveyor_resistances.resistance_secondary(q_v=q_v, p=p, v=v, v_0=v_0,
                                                B=B, b1=b1, mu1=mu1, mu2=mu2,
                                                wrap_a_h=wrap_a, wrap_a_t=wrap_a,
                                                f_1t_d=f_1t_d, f_1t_t=f_1t_t)
print(f"f_n:\t{f_n:.0f} N")
# f_u: Peripheral driving force
f_u = f_h + f_n + f_s
print(f"f_u:\t{f_u:.0f} N")
# p_m: Power at drive motor
p_m = power_requirements.power_requirements_motor(f_u=f_u, v=v, d_eta_1=d_eta_1, d_eta_2=d_eta_2) / 1000
print(f"p_m:\t{p_m:.2f} kW")

f_n:	5751 N
f_u:	12806 N
p_m:	66.71 kW
